<hr>
<center><h1>RSA : Rivest-Shamir-Adleman</h1></center>
<hr>

In [13]:
import hashlib
import random

## RSA Parameters

In [38]:
# Generate two 1024-bit primes for 2048-bit RSA
p = random_prime(2^1024, False, 2^1023)
q = random_prime(2^1024, False, 2^1023)
while q == p:
    q = random_prime(2^1024, False, 2^1023)

n = p * q
phi = (p - 1) * (q - 1)
e = 65537
d = inverse_mod(e, phi)

print(f"RSA-2048 generated")
print(f"n bits: {n.nbits()}")
print(f"e = {e}")

RSA-2048 generated
n bits: 2048
e = 65537


## 1. KeyGen

In [39]:
# Public key: (n, e)
public_key = (n, e)
# Private key: (n, d)
private_key = (n, d)

print(f"Public key (n, e): {public_key}")
print(f"Private key (n, d): {private_key}")

Public key (n, e): (23485422462297094589610529260787626662073673695723400283535576879941785215069523917125024962460942491215148099288996201389633024346553181477444886512266147635881172072714913131788860569879138949011811857580697945178926512140741379475978205196963672083956425332460575869424123661516988350846954584417582968101050204174214065321389856452130826072854768388317305836704388057942007840183142078801898652894118377843500115504568538800587037894152547975577087120510637066038700081834032644736287808647497049853367965170248948668460463794784304406945587801769146896136750174203703031233543533728573503607317146302651716418463, 65537)
Private key (n, d): (234854224622970945896105292607876266620736736957234002835355768799417852150695239171250249624609424912151480992889962013896330243465531814774448865122661476358811720727149131317888605698791389490118118575806979451789265121407413794759782051969636720839564253324605758694241236615169883508469545844175829681010502041742140653213898564

## 2. Encryption/Decryption

In [41]:
# Message to encrypt
message = "Hello RSA!"
print(f"Original message: {message}")

# Convert message to integer
m_bytes = message.encode('utf-8')
m = int.from_bytes(m_bytes, byteorder='big')
print(f"Message as integer: {m}")

# Check if message is too large for n
if m >= n:
    print(f"Warning: Message too large for n={n}")
    print("In practice, use hybrid encryption or larger primes")
else:
    # Encrypt: c = m^e mod n
    c = pow(m, e, n)
    c = int(c)  
    print(f"Encrypted (ciphertext): {c}")

    # Decrypt: m_dec = c^d mod n
    m_dec = pow(c, d, n)
    m_dec = int(m_dec)  
    print(f"Decrypted as integer: {m_dec}")

    # Convert back to string
    byte_length = (m_dec.bit_length() + 7) // 8
    decrypted_message = m_dec.to_bytes(byte_length, byteorder='big').decode('utf-8')
    print(f"Decrypted message: {decrypted_message}")
    print(f"Success: {message == decrypted_message}")

Original message: Hello RSA!
Message as integer: 341881320659697311891745
Encrypted (ciphertext): 4386609575850195519391104761119910922849445416861576985618260027720206729313228718804218392711200627688306940168638264940585950255067426894661345877276757684425775297620206922598344078064185675953792238433766977610206333355076651741143953108774667396891969575201865443204909986355523762741106405264687160386352113893162279140203471116606081297838279825879241899204925365260976573135996257519648317476536614870961634133207913843626888043880413919161947765611761720243547733973653207378831294106125083387826822939537466263888355128654906516596099779884687411860795464894507469175858803057505671308553598946425413359281
Decrypted as integer: 341881320659697311891745
Decrypted message: Hello RSA!
Success: True


## 3. Sign

In [42]:
# Message to sign
m = "RSA Signature"
print(f"Message: {m}")

# Hash the message
hash_obj = hashlib.sha256(m.encode('utf-8'))
hash_bytes = hash_obj.digest()
hash_int = int.from_bytes(hash_bytes, byteorder='big')

# Ensure hash is less than n
hash_int = hash_int % n
print(f"Hash as integer: {hash_int}")

# Sign: s = hash^d mod n (using private key)
signature = pow(hash_int, d, n)
print(f"Signature: {signature}")

Message: RSA Signature
Hash as integer: 306293111219355678564889886130685538039237078670802117338775761979186504898
Signature: 97376690641458230792669870393983014533721217254638143046854304922391402662284929492215588895169850031334334499584731968439195669594328384476877514036720883031560758696559228671949529589332113979576794881563474667074183234545534099718738623746426064114409532412186144544944076508234998032300044689780456979060309483574369877055587354355478824419290529341470356646200222586456483178167827076536164118336204012301796533228661325731397505432484665769983982318065018522943355523915352229447343456603399095738458736131717677096491514570688114725275447455291807369061890772504547879838782061890337900353208274697564192017


## 4. Verify

In [43]:
# Message and signature to verify
m = "RSA Signature"
s = signature

# Hash the message
hash_obj = hashlib.sha256(m.encode('utf-8'))
hash_bytes = hash_obj.digest()
hash_int = int.from_bytes(hash_bytes, byteorder='big') % n

# Verify: hash' = s^e mod n (using public key)
hash_prime = pow(s, e, n)

valid = (hash_int == hash_prime)
print(f"Original hash: {hash_int}")
print(f"Recovered hash: {hash_prime}")
print(f"Signature valid: {valid}")

Original hash: 306293111219355678564889886130685538039237078670802117338775761979186504898
Recovered hash: 306293111219355678564889886130685538039237078670802117338775761979186504898
Signature valid: True


## 5. RSA-CRT for faster decryption

In [48]:

p_int = int(p)
q_int = int(q)
n_int = int(n)
e_int = int(e)
d_int = int(d)
cipher_int = int(cipher)

# Decryption (CRT)
dP = d_int % (p_int - 1)
dQ = d_int % (q_int - 1)
qInv = int(inverse_mod(q_int, p_int))

# Convert results to Python ints immediately
m1 = int(pow(cipher_int, dP, p_int))  
m2 = int(pow(cipher_int, dQ, q_int))  

# Now both are Python ints, subtraction works
h = (qInv * (m1 - m2)) % p_int
m_crt = m2 + h * q_int

# Use original message byte length
msg_bytes = msg.encode('utf-8')
byte_length = len(msg_bytes)
dec_msg_crt = m_crt.to_bytes(byte_length, 'big').decode()
print(f"5. CRT decryption: m = '{dec_msg_crt}'")
print(f"   Match: {msg == dec_msg_crt}")

# Fast decryption using CRT
def rsa_decrypt_crt(c, p, q, dP, dQ, qInv):
    c_int = int(c)
    m1 = int(pow(c_int, dP, p))
    m2 = int(pow(c_int, dQ, q))
    h = (qInv * (m1 - m2)) % p
    return int(m2 + h * q)

# Test with our ciphertext
if 'cipher' in locals() and 'msg_int' in locals() and msg_int < n:
    m_crt_test = rsa_decrypt_crt(cipher, p_int, q_int, dP, dQ, qInv)
    dec_int = int(pow(cipher_int, d_int, n_int))
    print(f"\nCRT Decryption test: {m_crt_test}")
    print(f"Matches regular decryption: {m_crt_test == dec_int}")

5. CRT decryption: m = 'Hi'
   Match: True

CRT Decryption test: 18537
Matches regular decryption: True


## Complete RSA Demonstration

In [49]:
print("=== RSA COMPLETE DEMO ===\n")

# Key Generation
print("1. Key Generation:")
print(f"   p = {p}, q = {q}")
print(f"   n = p * q = {n}")
print(f"   φ(n) = (p-1)*(q-1) = {phi}")
print(f"   e = {e} (public exponent)")
print(f"   d = {d} (private exponent)\n")

# Message - choose one that's less than n
msg = input 
print(f"2. Original message: '{msg}'")
msg_bytes = msg.encode('utf-8')
byte_length = len(msg_bytes)  
msg_int = int.from_bytes(msg_bytes, 'big')
print(f"   As integer: {msg_int} (bytes: {byte_length}, must be < n={n})\n")

# Encryption
cipher = pow(msg_int, e, n)
cipher = int(cipher)  
print(f"3. Encryption: c = m^e mod n = {cipher}\n")

# Decryption (regular)
dec_int = pow(cipher, d, n)
dec_int = int(dec_int)
dec_msg = dec_int.to_bytes(byte_length, 'big').decode()  
print(f"4. Regular decryption: m = c^d mod n = '{dec_msg}'\n")

# Decryption (CRT) - Convert everything to Python ints
p_int = int(p)
q_int = int(q)
d_int = int(d)
cipher_int = int(cipher)

dP = d_int % (p_int - 1)
dQ = d_int % (q_int - 1)
qInv = int(inverse_mod(q_int, p_int))

m1 = int(pow(cipher_int, dP, p_int))
m2 = int(pow(cipher_int, dQ, q_int))
h = (qInv * (m1 - m2)) % p_int
m_crt = int(m2 + h * q_int)

dec_msg_crt = m_crt.to_bytes(byte_length, 'big').decode()  
print(f"5. CRT decryption: m = '{dec_msg_crt}'")
print(f"   Match: {msg == dec_msg_crt}")

=== RSA COMPLETE DEMO ===

1. Key Generation:
   p = 168719334728372904459212573101949034224857143481945082604822361875333664422502016723022256861683217906420493233005755841743902602330135366337580586788186520093367711955600619402064918414959470529430785535334679910335382968323968505740355562200811387325796526752944232120252677944084309382448897631008661080311, q = 139198168959752531793370931739274769200659072931821182279562409681529325308260241963486832763519386719239882260018959482978163335723447827952159629441825806085271628527471326164755478116783083551574489631553046040566795150189883911766183467601418815734498148160090366997920498810063504359115982153210006446233
   n = p * q = 2348542246229709458961052926078762666207367369572340028353557687994178521506952391712502496246094249121514809928899620138963302434655318147744488651226614763588117207271491313178886056987913894901181185758069794517892651214074137947597820519696367208395642533246057586942412366151698835084695458441758296810